In [1]:
import pandas as pd
import numpy as np

In [23]:
df = pd.read_csv('Nov10_2022/simvision_fastcmd_fc_int.csv', skiprows=2)

In [24]:
df.head()

,3000001000,1
0,3007681000,0
1,3020161000,1
2,3032641000,0
3,3045121000,1
4,3057601000,0


In [25]:
### Original column names
### 'SimTime' 
### 'ETROC2_tb.FCGenLocal_inst.FastComByte[7:0]'
### FastComByte

### Replace column names (only when fast command file has been opened)
df.columns = ['SimTime', 'FastComByte']
df = df.iloc[:-1 , :] ## drop the last row
df.head(10)

,SimTime,FastComByte
0,3007681000,0
1,3020161000,1
2,3032641000,0
3,3045121000,1
4,3057601000,0
5,3070081000,1
6,3082561000,0
7,3095041000,1
8,3107521000,0
9,3120001000,1


In [26]:
sim = df.diff(periods=-1).fillna(0)
sim['SimTime'] = sim['SimTime'].abs()*0.001
sim['nth'] = sim['SimTime']/(1560*2)
sim.head(10)

,SimTime,FastComByte,nth
0,12480.0,-1.0,4.0
1,12480.0,1.0,4.0
2,12480.0,-1.0,4.0
3,12480.0,1.0,4.0
4,12480.0,-1.0,4.0
5,12480.0,1.0,4.0
6,12480.0,-1.0,4.0
7,12480.0,1.0,4.0
8,12480.0,-1.0,4.0
9,12480.0,1.0,4.0


In [27]:
df['diff'] = sim['SimTime'] ## Time difference between (nth, (n+1)th) row, n = 0, 1, 2, ...
df['cycle'] = sim['nth']
df.head(10)

,SimTime,FastComByte,diff,cycle
0,3007681000,0,12480.0,4.0
1,3020161000,1,12480.0,4.0
2,3032641000,0,12480.0,4.0
3,3045121000,1,12480.0,4.0
4,3057601000,0,12480.0,4.0
5,3070081000,1,12480.0,4.0
6,3082561000,0,12480.0,4.0
7,3095041000,1,12480.0,4.0
8,3107521000,0,12480.0,4.0
9,3120001000,1,12480.0,4.0


In [28]:
df.tail(10)

,SimTime,FastComByte,diff,cycle
80671,1007410561000,1,12480.0,4.0
80672,1007423041000,0,12480.0,4.0
80673,1007435521000,1,12480.0,4.0
80674,1007448001000,0,12480.0,4.0
80675,1007460481000,1,12480.0,4.0
80676,1007472961000,0,12480.0,4.0
80677,1007485441000,1,12480.0,4.0
80678,1007497921000,0,12480.0,4.0
80679,1007510401000,1,12480.0,4.0
80680,1007522881000,0,0.0,0.0


In [29]:
### Repeat rows in a pandas DataFrame based on column value
### https://stackoverflow.com/questions/47336704/repeat-rows-in-a-pandas-dataframe-based-on-column-value
bit_df = df.reindex(df.index.repeat(df.cycle))
bit_df = bit_df.drop(columns=['SimTime','diff', 'cycle'])
bit_df = bit_df.reset_index(drop=True)
bit_df.head(25)

,FastComByte
0,0
1,0
2,0
3,0
4,1
5,1
6,1
7,1
8,0
9,0


In [30]:
subset1 = []

output = []
for i in range(int(bit_df.shape[0]/8)):
#for i in range(80):
    x1, x2 = 0+8*i, 8+8*i
    fcmd_bit = bit_df.iloc[range(x1, x2), :].values.flatten().tolist()
    fcmd_bit.reverse() ## Raw data bit order is reversed!
    s1 = ''.join([str(x) for x in fcmd_bit])
    output.append([s1, format(int(s1, 2), 'X')])
    
    if i < 10:
        print(s1, format(int(s1, 2), 'X'))

11110000 F0
11110000 F0
11110000 F0
11110000 F0
11110000 F0
11110000 F0
11110000 F0
11110000 F0
11110000 F0
11110000 F0


In [31]:
from datetime import date
today = str(date.today())

new_df = pd.DataFrame(columns=['fastcmd_8bit', 'fastcmd_hex'], data=output)
new_df.to_csv('output/fastcmd_result_test'+today+'.csv', index=False)